## 掛載雲端硬碟

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 更改檔案所在路徑


In [2]:
# Change to your own folder !!!
%cd /content/drive/MyDrive/Colab-AILAB/LAB2-Model_Pruning/

/content/drive/MyDrive/Colab-AILAB/LAB2-Model_Pruning


## 載入函式庫


In [3]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np

from models.resnet import ResNet50

## 超參數設定

In [4]:
DATASET = 'cifar10'
TEST_BATCH_SIZE = 1000
CUDA = True
PRUNE_PERCENT = 0.9 # Change your prune ratio!
WEIGHT_PATH = '/content/drive/MyDrive/Colab-AILAB/LAB2-Model_Pruning/model_best.pth' # Change to your own folder !!!
PRUNE_PATH = '/content/drive/MyDrive/Colab-AILAB/LAB2-Model_Pruning/model_prune.pth' # Change to your own folder !!!

## 載入模型

In [5]:
CUDA = CUDA and torch.cuda.is_available()

model = ResNet50(num_classes=10)
if CUDA:
    model.cuda()

if WEIGHT_PATH:
    if os.path.isfile(WEIGHT_PATH):
        checkpoint = torch.load(WEIGHT_PATH)
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print('LOADING CHECKPOINT {} @EPOCH={}, BEST_PREC1={}'.format(WEIGHT_PATH,checkpoint['epoch'],best_prec1))

    else:
        print("NO CHECKPOINT FOUND")

print(model)

LOADING CHECKPOINT /content/drive/MyDrive/Colab-AILAB/LAB2-Model_Pruning/model_best.pth @EPOCH=40, BEST_PREC1=0.9053999781608582
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

## 進行剪枝
#### 計算所有Batch Normalizaiton中的scale factor絕對值大小並排序
#### 利用設定好的PRUNE_PERCENT來取得閥值

In [6]:
total = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        total += m.weight.data.shape[0]

bn = torch.zeros(total)
index = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        size = m.weight.data.shape[0]
        bn[index:(index+size)] = m.weight.data.abs().clone()
        index += size

y, i = torch.sort(bn)

threshold_index = int(total * PRUNE_PERCENT)
threshold = y[threshold_index]


## 根據Batch Normalization Layer資訊建立CONFIG
#### 1. 複製Batch Normalization Layer的weight(也就是scale factor γ)
#### 2. 建立mask，大於threshold的index的值會設成1,小於threshold的值會設成0
#### 3. mask的值加總後，會是剪枝後Layer對應的輸出channel數
#### 4. 最後得到要建立剪枝模型的CONFIG

In [7]:
pruned = 0
cfg = []  #用來建立剪枝網路的CONFIG
cfg_mask = [] #用來幫助剪枝的遮罩

In [8]:
bn_layer_counter = 0
for k, m in enumerate((model.modules())):
    if isinstance(m, nn.BatchNorm2d):
        bn_layer_counter += 1 # BN 層計數
        weight_copy = m.weight.data.clone()
        #mask = weight_copy.abs().gt(threshold).float().cuda() # 大於 threshold 的設為 True (1.0)，其餘為 False(0.0)

        # 注意: 需自行設計處理剩下channel數為0的情況 (e.g. 至少保留3個channel)
        ################################################
        #          請填空          #
        ################################################

        is_bn3 = (bn_layer_counter > 1) and ((bn_layer_counter - 1) % 3 == 0)
        if is_bn3:
            mask = torch.ones(weight_copy.shape[0]).float().cuda()
        else: #做prune
            mask = weight_copy.abs().gt(threshold).float().cuda() # 大於 threshold 的設為 True (1.0)

            #至少要保留三個channel
            k_min = min(3, mask.shape[0]) # 至少保留 k_min 個
            if torch.sum(mask) < k_min:
                # 找出 gamma 絕對值前 k_min 大的索引
                _, topk_indices = torch.topk(weight_copy.abs(), k_min)
                # 先清零，再把 top-k 設為 1.0
                mask.zero_()
                mask[topk_indices] = 1.0

        # 處理剪枝後的權重
        m.weight.data.mul_(mask)
        m.bias.data.mul_(mask)
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        cfg.append(int(torch.sum(mask)))    # 記錄每一層 BN 剩下幾個通道
        cfg_mask.append(mask.clone())     # 儲存每層對應的 mask
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))

pruned_ratio = pruned/total

print(f'PRUNE RATIO={pruned_ratio}')
print('PREPROCESSING SUCCESSFUL!')

print(f'cfg: {cfg}')


layer index: 2 	 total channel: 64 	 remaining channel: 64
layer index: 8 	 total channel: 64 	 remaining channel: 64
layer index: 10 	 total channel: 64 	 remaining channel: 64
layer index: 12 	 total channel: 256 	 remaining channel: 256
layer index: 18 	 total channel: 64 	 remaining channel: 64
layer index: 20 	 total channel: 64 	 remaining channel: 64
layer index: 22 	 total channel: 256 	 remaining channel: 256
layer index: 26 	 total channel: 64 	 remaining channel: 64
layer index: 28 	 total channel: 64 	 remaining channel: 64
layer index: 30 	 total channel: 256 	 remaining channel: 256
layer index: 35 	 total channel: 128 	 remaining channel: 118
layer index: 37 	 total channel: 128 	 remaining channel: 123
layer index: 39 	 total channel: 512 	 remaining channel: 512
layer index: 45 	 total channel: 128 	 remaining channel: 117
layer index: 47 	 total channel: 128 	 remaining channel: 126
layer index: 49 	 total channel: 512 	 remaining channel: 512
layer index: 53 	 total 

## 建立剪枝模型

In [9]:
newmodel = ResNet50(num_classes=10, cfg=cfg)
newmodel.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 將原本的模型權重複製到剪枝的模型
#### 根據不同層決定要複製什麼權重
###### Batch Normalization Layer
1.   scale factor
2.   bias
3.   running mean
4.   running variance

###### Convolutional Layer
1.   weight

###### Linear Layer
1.   weight
2.   bias




In [10]:
old_modules = list(model.modules())
new_modules = list(newmodel.modules())

layer_id_in_cfg = 0
start_mask = torch.ones(3) #3為input channel(R,G,B)
end_mask = cfg_mask[layer_id_in_cfg]
bn_count = 0

for layer_id in range(len(old_modules)):

    m0 = old_modules[layer_id] #雖然說有mask過但是實際結構仍然和原本的一樣,他還是會存在,所以要複製到新的讓他真正有做到pruning
    m1 = new_modules[layer_id] #把1.0,也就是mask過有保留的channel留下來，讓整體的結構縮小，達到Channel Pruning

    if isinstance(m0, nn.BatchNorm2d):
        bn_count += 1
        #### 找出遮罩中非零元素的index ####
        ################################################
        #          請填空          #
        ################################################

        # np.argwhere 會找出 end_mask 中不為0的indices e.g.[0,1,0,1,1] -> 回傳{[1],[3],[4]} -> shape=(3,1)
        # np.squeeze 會把dimention=1的remove
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.ndim == 0:
            idx1 = np.expand_dims(idx1, 0)
        # 這一部分主要是避免如果只剩一個channel np.squeeze()會讓他變成純量，不過前面就已經至少保留三個channel所以基本上不會用到這個

        #### 複製weight, bias, running mean,and running variance ####
        ################################################
        #          請填空          #
        ################################################

        #把weight bias running_mean running_variance複製到過去(mask過後的複製過去)
        m1.weight.data = m0.weight.data[idx1].clone()
        m1.bias.data = m0.bias.data[idx1].clone()
        m1.running_mean = m0.running_mean[idx1].clone()
        m1.running_var = m0.running_var[idx1].clone()

        layer_id_in_cfg += 1
        start_mask = end_mask.clone()
        if layer_id_in_cfg < len(cfg_mask):
            end_mask = cfg_mask[layer_id_in_cfg]


    elif isinstance(m0, nn.Conv2d): #Conv weight長[out_channels,in_channels,k,k]
        if isinstance(old_modules[layer_id + 1], nn.BatchNorm2d):
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy()))) #來自start_mask,要保留的in_channel indices
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy()))) #來自end_mask,要保留out_channel indices

            #### 複製weight ####
            ################################################
            #          請填空          #
            ################################################

            # 一樣是為了避免他squeeze後會變成純量導致程式BUG，不過其實他也不會進來執行，因為本來就已經至少保留三個channel
            if idx0.ndim == 0:
                idx0 = np.expand_dims(idx0, 0)
            if idx1.ndim == 0:
                idx1 = np.expand_dims(idx1, 0)

            w1 = m0.weight.data[idx1, :, :, :].clone() #用idx1選出out_channels -> dim0
            m1.weight.data = w1[:, idx0, :, :].clone() #用idx0選出in_channels -> dim1


        # downsample 層不用prune
        else:
            m1.weight.data = m0.weight.data.clone()


    elif isinstance(m0, nn.Linear): #FC layer長 [out_features,in_features]

        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy()))) #come from start_mask, is in_features
        if idx0.ndim == 0:
            idx0 = np.expand_dims(idx0, 0)

        #### 複製weight ####
        ################################################
        #          請填空          #
        ################################################

        # 保留所有的out_features
        # 只剪in_features
        m1.weight.data = m0.weight.data[:, idx0].clone()

        #### 複製bias ####
        m1.bias.data = m0.bias.data.clone()


## 測試函數




In [11]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
        batch_size=TEST_BATCH_SIZE, shuffle=True, **kwargs)

    model.eval()
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
          if CUDA:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

## 儲存模型並印出結果，以及剪枝後的test acc


In [12]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()}, PRUNE_PATH)

print(newmodel)
model = newmodel.cuda()
test(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

tensor(0.6326)